In [1]:
MutationFile = 'Cutaneous_somatic_mutation_MELA_AU_DNVs_sorted_dinuc_filter.bed'
FastaFile = 'hg19.fa'
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'

from SequenceContext import CreateNewBedFile
CreateNewBedFile(1,1,MutationFile,FastaFile)

from CompareMutationContext import AttainMutationData
AttainMutationData('AC>TT', OutputFile)



In [ ]:
from ActuallyNormalizeData import NormalizeData
NormalizeData(OutputFile, MutationFile, 4)


In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
gdata = pd.read_csv(OutputFile, header = None, names= ['Mutation' , 'Frequency'])
sns.barplot(x = 'Mutation', y = 'Frequency', data = gdata,)